# 50인 데이터 매칭 알고리즘

사용 언어: Python

가치관 질문은 사용하지 않는다고 가정하고 우선 제외한 상태로 작성했습니다.

현재 구현 버전에서는 성별/연령 차이를 구현하지 않았습니다.

(09/09 수정: 매칭 알고리즘 보완)

(09/15 수정: 매칭 알고리즘 재수정, 2인씩 매칭하도록 고침. 전체 응답차 내에서 순위를 정해서 내려오는 방식으로 수정, 성별 및 연령 도입)

(10/07 수정: 전화번호로 매칭하기 구현, 가장 비슷한 문항 / 가장 다른 문항 함께 출력하기)

## 데이터 정제

데이터 분석 및 정리에 필요한 라이브러리(pandas, numpy)를 불러옵니다.

주어진 엑셀 파일에서 미리 사용하지 않을 열(타임스탬프, 가치관 질문, 빈 열 등)을 삭제하고, CSV 파일형식으로 변환하여 불러옵니다.
dataCount 변수에 현재 데이터 개수(응답 인원수)를 저장합니다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# set of column names
headerNames = ["phoneNum","q1","q2", "q3", "q4", "q5", "q6", "q7","q8", "q9", "q10", "gender", "age", "polAlign"]

# read csv-converted data from excel file
# current data is cleaned to exclude empty and/or meaningless columns
answerData = pd.read_csv('data/50data_phonenum.csv', header =0, names=headerNames)

# save number of entries
dataCount = len(answerData.index)

비수치화 되어 있는 응답 결과를 숫자 형태의 데이터값으로 변환해 줍니다.

In [3]:
# replace all redundant text to interger values

answerData.replace(["① 매우 찬성","② 찬성", "③ 중립", "④ 반대", "⑤ 매우 반대", "남", "여"], [1, 2, 3, 4, 5, "M", "F"], inplace = True)

전화번호로 분류된 데이터가 들어갔을 때

In [4]:
answerData

,phoneNum,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,gender,age,polAlign
0,010-0000-0000,3,1,2,4,2,1,3,2,3,2,M,40,2
1,010-0000-0001,1,3,2,2,5,4,2,4,3,1,F,25,2
2,010-0000-0002,2,5,3,2,1,4,2,3,5,2,M,31,3
3,010-0000-0003,5,2,4,5,1,4,1,1,5,3,M,51,4
4,010-0000-0004,3,3,3,3,2,3,3,3,3,3,F,38,3
5,010-0000-0005,2,4,2,4,2,4,2,4,2,3,F,47,3
6,010-0000-0006,1,4,2,3,5,1,5,2,1,2,F,21,2
7,010-0000-0007,2,1,2,2,4,3,3,2,3,1,M,32,3
8,010-0000-0008,4,2,5,4,4,4,4,1,1,1,F,52,4
9,010-0000-0009,5,3,5,5,1,2,2,2,4,1,M,56,5


## 응답 유사도 측정

모든 개인과 개인 간의 응답 유사도를 저장할 (데이터 수)^2 사이즈의 행렬인 diffMatrix를 새로 만듭니다.
기본적으로는 NaN (값이 없는 상태)로 초기화합니다.

In [5]:
# create a new dataframe to save values
# this will be very time and cost extensive, might be viable to change this part
# will be durable for several hundred calculations though...?

diffMatrix = pd.DataFrame(np.nan, index=answerData.phoneNum, columns=answerData.phoneNum, dtype='float')

각 개인과 개인의 응답 결과를 비교하여, 각 질문에 대한 총 응답 차이값 = Sum(|개별 응답 차|) 값을 저장합니다.
i번째 응답자와 j번째 응답자의 응답 차이값은 diffMatrix의 i행 j열에 저장됩니다. (j행 i열의 값과 동일합니다.)
자기 자신과의 응답차는 계산하지 않도록 짜 두어, 대각선으로 NaN 값이 나옵니다.

(50인 데이터에서는 수 초 내에 결과가 나오지만, 실제 데이터에서는 응답차 값을 계산하는 것에 상당한 시간이 소요될 것으로 예상됩니다.
 현재 모델에서는 개인과 개인 간의 총 응답차 값을 모두 계산해야 하기 때문에 이 부분이 필수적이라는 점 또한 유의해 주시기 바랍니다.)
 
(09/15 수정: 연령 및 성별 차이를 반영하였습니다. 응답차가 최우선적으로 반영되지만, 성별/연령이 순차적으로 반영될 수 있게 성별은 0.1값, 연령은 0.001값으로 계산합니다.)

In [6]:
for i in range(dataCount):
    for j in range(i, dataCount):
        tempVal=0;
        if (i==j):
            diffMatrix.iloc[i][j] = np.nan
            continue
        for k in range(1,11):
            # iterates through 10 for the number of questions in the dataframe
            tempVal += abs(answerData.iloc[i][k]-answerData.iloc[j][k])
            
        tempVal = abs(tempVal)
        # assign values for gender/age difference
        if answerData.iloc[i]["gender"] != answerData.iloc[j]["gender"]:
            tempVal += 0.1
            
        tempVal += abs(answerData.iloc[i]["age"]-answerData.iloc[j]["age"])*0.001
        
        # Assign same value to the flipped index to save computation time
        diffMatrix.iloc[i][j] = tempVal
        diffMatrix.iloc[j][i] = tempVal

In [7]:
diffMatrix

phoneNum,010-0000-0000,010-0000-0001,010-0000-0002,010-0000-0003,010-0000-0004,010-0000-0005,010-0000-0006,010-0000-0007,010-0000-0008,010-0000-0009,...,010-0000-0040,010-0000-0041,010-0000-0042,010-0000-0043,010-0000-0044,010-0000-0045,010-0000-0046,010-0000-0047,010-0000-0048,010-0000-0049
phoneNum,,,,,,,,,,,,,,,,,,,,,
010-0000-0000,NaN,16.115,16.009,16.011,8.102,12.107,13.119,8.008,15.112,13.016,...,7.005,24.025,21.016,7.117,10.108,12.118,9.020,17.001,9.015,17.005
010-0000-0001,16.115,NaN,12.106,22.126,12.013,10.022,13.004,8.107,17.027,19.131,...,15.120,20.140,19.131,15.032,18.023,16.033,11.105,15.114,13.100,23.120
010-0000-0002,16.009,12.106,NaN,14.020,10.107,10.116,19.110,14.001,21.121,15.025,...,15.014,20.034,23.025,15.126,18.117,16.127,11.011,17.008,15.006,17.014
010-0000-0003,16.011,22.126,14.020,NaN,14.113,16.104,27.130,20.019,15.101,9.005,...,11.006,20.014,17.005,17.106,20.103,14.107,15.031,21.012,21.026,11.006
010-0000-0004,8.102,12.013,10.107,14.113,NaN,8.009,15.017,10.106,15.014,13.118,...,5.107,18.127,21.118,7.019,16.010,10.020,1.118,11.101,13.113,17.107
010-0000-0005,12.107,10.022,10.116,16.104,8.009,NaN,15.026,14.115,17.005,17.109,...,11.102,18.118,23.109,13.010,18.001,14.011,9.127,9.108,13.122,23.102
010-0000-0006,13.119,13.004,19.110,27.130,15.017,15.026,NaN,13.111,16.031,22.135,...,16.124,21.144,24.135,14.036,15.027,19.037,14.101,12.118,12.104,24.124
010-0000-0007,8.008,8.107,14.001,20.019,10.106,14.115,13.111,NaN,13.120,17.024,...,9.013,24.033,19.024,7.125,14.116,12.126,9.012,15.007,9.007,17.013
010-0000-0008,15.112,17.027,21.121,15.101,15.014,17.005,16.031,13.120,NaN,14.104,...,10.107,17.113,14.104,10.005,21.004,15.006,14.132,14.113,16.127,12.107


diffMatrix 내에 저장되어 있는 모든 값들의 중간값을 diffMedian 변수에 저장합니다.
50인 테스트 데이터셋의 경우, 중간값은 5.111에서 형성되었습니다.

In [8]:
# Calculate the median value of all the diff values (currently, it's 5)
diffMedian = diffMatrix.stack().median()

print(diffMedian)

16.019


## 매칭 알고리즘 - 전체 매칭: 1인 기준 (09/15 추가) 

위와 동일한 결과를 기준으로, 같은생각/다른생각에 대하여 각 1인씩만 매칭하는 알고리즘으로 변형시켜 보았습니다.

(앞선 과정과 유사하거나 같은 기능을 하는 변수들은 _all_1 을 붙여서 구분합니다)

In [9]:
samePairs_all_1 = []

# Save the number of times each index has been matched
matchSame_all_1 = [0]*dataCount

# Copy the entire diffMatrix values
sameCount_matrix_1 = diffMatrix.to_numpy(copy = True)

for i in range(dataCount):
    sameCount_matrix_1[i][i] = 100

while np.min(sameCount_matrix_1) < diffMedian and np.sum(matchSame_all_1) < dataCount:
    # calculate the indices of the current minimun value
    tempInd = np.unravel_index(np.argmin(sameCount_matrix_1), (dataCount, dataCount))
    
    if matchSame_all_1[tempInd[0]] >= 1 or matchSame_all_1[tempInd[1]] >= 1:
        sameCount_matrix_1[tempInd[0],tempInd[1]] = 100
        continue
    
    tempRank_0 = (sameCount_matrix_1[tempInd[0]] == 100).sum() 
    tempRank_1 = (sameCount_matrix_1[tempInd[1]] == 100).sum() 
    
    tempPairValue = [answerData.phoneNum[tempInd[0]], answerData.phoneNum[tempInd[1]], tempRank_0, tempRank_1]
    
    tempMin = 10
    tempMinArray = []
    for j in range(1,11):
        tempDiff = abs(answerData.iloc[tempInd[0]][j]-answerData.iloc[tempInd[1]][j])
        if (tempDiff < tempMin):
            tempMinArray = [j]
            tempMin = tempDiff
        elif(tempDiff == tempMin):
            tempMinArray.append(j)
            
    tempPairValue.append(tempMinArray)
    
    samePairs_all_1.append(tempPairValue)
    sameCount_matrix_1[tempInd[0],tempInd[1]] = 100
    sameCount_matrix_1[tempInd[1],tempInd[0]] = 100
    
    # increment counts 
    matchSame_all_1[tempInd[0]] += 1
    matchSame_all_1[tempInd[1]] += 1
    
            
print(samePairs_all_1)

[['010-0000-0004', '010-0000-0024', 1, 1, [1, 2, 3, 4, 5, 6, 7, 8, 9]], ['010-0000-0014', '010-0000-0017', 1, 1, [1, 2, 3, 4, 6, 7, 8, 9]], ['010-0000-0016', '010-0000-0025', 1, 1, [1, 2, 3, 4, 5, 8, 10]], ['010-0000-0003', '010-0000-0015', 1, 1, [1, 4, 5, 6, 7, 8, 9]], ['010-0000-0037', '010-0000-0039', 1, 1, [1, 3, 4, 5, 6, 8, 9, 10]], ['010-0000-0020', '010-0000-0046', 1, 3, [1, 3, 5, 7, 8, 10]], ['010-0000-0019', '010-0000-0027', 1, 1, [2, 5, 6, 8]], ['010-0000-0000', '010-0000-0040', 2, 4, [1, 4, 7, 9]], ['010-0000-0038', '010-0000-0047', 1, 1, [1, 3, 4]], ['010-0000-0023', '010-0000-0043', 1, 3, [1, 2, 3, 10]], ['010-0000-0030', '010-0000-0048', 1, 1, [4, 6, 9]], ['010-0000-0009', '010-0000-0011', 2, 1, [2, 3, 5, 7, 10]], ['010-0000-0007', '010-0000-0022', 2, 2, [2, 4, 6, 7, 10]], ['010-0000-0002', '010-0000-0031', 1, 1, [2, 3, 5, 6, 9]], ['010-0000-0033', '010-0000-0036', 1, 2, [1, 2, 7, 8, 9]], ['010-0000-0010', '010-0000-0028', 1, 3, [1, 2, 4, 6, 7]], ['010-0000-0006', '010-00

다른생각 매칭 또한 같은 원리로 진행했습니다.

In [10]:
diffPairs_all_1 = []

# Save the number of times each index has been matched
matchDiff_all_1 = [0]*dataCount

# Copy the entire diffMatrix values
diffCount_matrix_1 = diffMatrix.to_numpy(copy = True)

for i in range(dataCount):
    diffCount_matrix_1[i][i] = -100

while np.max(diffCount_matrix_1) > diffMedian and np.sum(matchDiff_all_1) < 1*dataCount:
    # calculate the indices of the current minimun value
    tempInd = np.unravel_index(np.argmax(diffCount_matrix_1), (dataCount, dataCount))
    
    if matchDiff_all_1[tempInd[0]] >= 1 or matchDiff_all_1[tempInd[1]] >= 1:
        diffCount_matrix_1[tempInd[0],tempInd[1]] = -100
        continue
    
    tempRank_0 = (diffCount_matrix_1[tempInd[0]] == -100).sum() 
    tempRank_1 = (diffCount_matrix_1[tempInd[1]] == -100).sum() 
    
    tempPairValue = [answerData.phoneNum[tempInd[0]], answerData.phoneNum[tempInd[1]], tempRank_0, tempRank_1]
    
    tempMax = 0
    tempMaxArray = []
    for j in range(1,11):
        tempDiff = abs(answerData.iloc[tempInd[0]][j]-answerData.iloc[tempInd[1]][j])
        if (tempDiff > tempMax):
            tempMaxArray = [j]
            tempMax = tempDiff
        elif(tempDiff == tempMax):
            tempMaxArray.append(j)
            
    tempPairValue.append(tempMaxArray)
    
    diffPairs_all_1.append(tempPairValue)
    diffCount_matrix_1[tempInd[0],tempInd[1]] = -100
    diffCount_matrix_1[tempInd[1],tempInd[0]] = -100
    
    # increment counts 
    matchDiff_all_1[tempInd[0]] += 1
    matchDiff_all_1[tempInd[1]] += 1
    
            
print(diffPairs_all_1)

[['010-0000-0034', '010-0000-0037', 1, 1, [1, 3, 4, 5, 6, 7, 8, 9]], ['010-0000-0030', '010-0000-0041', 1, 1, [1, 2, 3, 7, 8]], ['010-0000-0015', '010-0000-0039', 2, 1, [1, 2, 3, 4, 5, 8, 9]], ['010-0000-0003', '010-0000-0013', 3, 2, [1, 4, 5, 7, 9]], ['010-0000-0014', '010-0000-0031', 2, 1, [2, 7, 9]], ['010-0000-0032', '010-0000-0033', 3, 1, [3, 7, 8, 9]], ['010-0000-0042', '010-0000-0048', 2, 2, [4, 6, 9]], ['010-0000-0009', '010-0000-0017', 1, 5, [1, 3, 5]], ['010-0000-0047', '010-0000-0049', 1, 3, [9]], ['010-0000-0029', '010-0000-0035', 3, 1, [3, 9]], ['010-0000-0025', '010-0000-0038', 3, 1, [4, 5, 10]], ['010-0000-0008', '010-0000-0018', 1, 2, [3, 8, 9]], ['010-0000-0021', '010-0000-0026', 2, 1, [6, 8]], ['010-0000-0002', '010-0000-0010', 1, 3, [2, 9]], ['010-0000-0012', '010-0000-0023', 2, 1, [9]], ['010-0000-0044', '010-0000-0045', 8, 7, [1]], ['010-0000-0011', '010-0000-0028', 10, 10, [5]], ['010-0000-0005', '010-0000-0016', 7, 9, [2, 4, 7, 8]], ['010-0000-0001', '010-0000-00

In [13]:
np.savetxt("samepairs_phoneNum.tsv", samePairs_all_1, delimiter="\t", fmt="%s")
np.savetxt("diffPairs_phoneNum.tsv", diffPairs_all_1, delimiter="\t", fmt="%s")

매칭 결과를 확인하기 위해 표로 정리했습니다. 50인 데이터를 기준으로, 같은생각/다른생각 모두 매칭되지 않은 경우가 1명 존재합니다.

중간값 등의 도입으로 인해, 같은생각/다른생각 둘 중 한 가지 이상이 매칭되지 않는 경우가 다수 존재합니다.

In [12]:
matchData_1=pd.DataFrame(
    {'같은생각 매칭 수':matchSame_all_1, '다른생각 매칭 수':matchDiff_all_1, '합':np.add(matchSame_all_1,matchDiff_all_1)})

matchData_1

,같은생각 매칭 수,다른생각 매칭 수,합
0,1,1,2
1,1,1,2
2,1,1,2
3,1,1,2
4,1,0,1
5,1,1,2
6,1,1,2
7,1,0,1
8,1,1,2
9,1,1,2
